In [1]:
#We can go into our root file and see what Trees are availiable
%matplotlib inline
import sys, os
if __package__ is None:
    import sys, os
    sys.path.append(os.path.realpath("/data/shared/Software/"))
import numpy as np
import pandas as pd
import ntpath
import glob
import deepconfig

from keras.utils.visualize_util import plot
from IPython.display import Image, display

from CMS_Deep_Learning.utils.preprocessing import *
from CMS_Deep_Learning.utils.metrics import plot_history, print_accuracy_m
from CMS_Deep_Learning.utils.callbacks import OverfitStopping, SmartCheckpoint
from CMS_Deep_Learning.utils.archiving import *

from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Reshape, Activation, Dropout, Convolution2D, merge, Input, Flatten
from keras.callbacks import EarlyStopping

#Choose the GPU
dc = deepconfig.deepconfig(gpu='gpu0', backend='theano')

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN not available)

Couldn't import dot_parser, loading of dot files will not be possible.
using gpu0
using theano


In [2]:
#Set up all of our preprocessing parameters
observ_types = ['E/c', 'Px', 'Py', 'Pz', 'Charge', "PT_ET", "Eta", "Phi", "Dxy_Ehad_Eem"]
vecsize = len(observ_types)
sample_start = 0
#NOT USING VERY MANY SMAPLES JUST SO ILLUSTRATE
num_samples = 1000


object_profiles = [ObjectProfile("Electron",5),
                    ObjectProfile("MuonTight", 5),
                    ObjectProfile("Photon", 25),
                    ObjectProfile("MissingET", 1),
                    ObjectProfile("EFlowPhoton",1300, sort_columns=["PT_ET"], sort_ascending=False),  #max ~1300
                    ObjectProfile("EFlowNeutralHadron",1000, sort_columns=["PT_ET"], sort_ascending=False),  #max ~1000
                    ObjectProfile("EFlowTrack",1050, sort_columns=["PT_ET"], sort_ascending=False)]  #max ~1050

label_dir_pairs = \
            [   ("ttbar", "/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/"),
                ("wjet", "/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/")#,#
#                ("qcd", "/data/shared/Delphes/qcd_lepFilter_13TeV/pandas_unjoined/")
            ]

In [3]:
start = time.clock()
X_train, Y_train = preprocessFromPandas_label_dir_pairs(label_dir_pairs,
                                                        sample_start,
                                                        num_samples,
                                                        object_profiles,
                                                        observ_types)
print("Preprocess Elapse Time:", time.clock()-start)

Reading 1000 samples from '/data/shared/Delphes/ttbar_lepFilter_13TeV/pandas_unjoined/ttbar_lepFilter_13TeV_0.h5':
Mapping 5 Values/Sample from 'Electron'
Mapping 5 Values/Sample from 'MuonTight'
Mapping 25 Values/Sample from 'Photon'
Mapping 1 Values/Sample from 'MissingET'
Mapping 1300 Values/Sample from 'EFlowPhoton'
Mapping 1000 Values/Sample from 'EFlowNeutralHadron'
Mapping 1050 Values/Sample from 'EFlowTrack'
*Read 1000 Samples of 1000 in range(0, 1000)
--------------------------------------------------
Reading 1000 samples from '/data/shared/Delphes/wjets_lepFilter_13TeV/pandas_unjoined/wjets_lepFilter_13TeV_1.h5':
Mapping 5 Values/Sample from 'Electron'
Mapping 5 Values/Sample from 'MuonTight'
Mapping 25 Values/Sample from 'Photon'
Mapping 1 Values/Sample from 'MissingET'
Mapping 1300 Values/Sample from 'EFlowPhoton'
Mapping 1000 Values/Sample from 'EFlowNeutralHadron'
Mapping 1050 Values/Sample from 'EFlowTrack'
*Read 1000 Samples of 1000 in range(0, 1000)
-------------------

In [4]:
start= time.clock()
XY_to_CSV(X_train,Y_train, "TestCSVDir")
print("to_CSV Elapse Time:", time.clock()-start)

('to_CSV Elapse Time:', 24.816235)


In [5]:
start= time.clock()
X,Y = XY_from_CSV("TestCSVDir")
print("from_CSV Elapse Time", time.clock()-start)

('from_CSV Elapse Time', 79.17493999999999)


In [6]:
for x, x_train in zip(X,X_train):
    np.testing.assert_array_almost_equal(x,x_train)
for y, y_train in zip(Y,[Y_train]):
    np.testing.assert_array_almost_equal(y,y_train)
print("Everything read back correctly")

Everything read back correctly


In [7]:
start= time.clock()
XY_to_pickle(X_train,Y_train, "TestPickleDir")
print("to_pickle Elapse Time:", time.clock()-start)

('to_pickle Elapse Time:', 0.160775000000001)


In [8]:
start= time.clock()
X,Y = XY_from_pickle("TestPickleDir")
print("from_pickle Elapse Time:", time.clock()-start)

('from_pickle Elapse Time:', 0.8444429999999983)


In [9]:
for x, x_train in zip(X,X_train):
    np.testing.assert_array_almost_equal(x,x_train)
for y, y_train in zip(Y,[Y_train]):
    np.testing.assert_array_almost_equal(y,y_train)
print("Everything read back correctly")

Everything read back correctly
